In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [8]:
llm = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./.files/chapter1_1984.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = FAISS.from_documents(docs, cache_embeddings)
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
        ("human", "{question}")
    ]
)

chain = (
        {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
)


In [9]:
chain.invoke("Where dose Victory Mansion.")

AIMessage(content='Victory Mansions is located in London, the chief city of Airstrip One, which is one of the provinces of Oceania.')

In [10]:
chain.invoke("Describe Victory Mansion.")

AIMessage(content='Victory Mansion is a building with glass doors that Winston Smith enters on a bright cold day in April. The hallway smells of boiled cabbage and old rag mats. It has seven flights of stairs, and the lift is usually not working due to the electricity being cut off during daylight hours. Inside the flat, there is a telescreen that cannot be completely shut off, and a poster with an enormous face that reads "BIG BROTHER IS WATCHING YOU."')